In [1]:
import pandas as pd
import os
import glob

os.chdir('C:/Users/natas/Downloads/safety/safety/features/')
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "df.csv", index=False, encoding='utf-8-sig')

In [2]:
import pandas as pd
import os
import glob

In [3]:
df_label = pd.read_csv("C:/Users/natas/Downloads/safety/safety/labels/part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv")

In [4]:
df= pd.read_csv('C:/Users/natas/Downloads/safety/safety/features/df.csv')

In [5]:
df = pd.merge(df, df_label, on='bookingID')


In [6]:
df.head()

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed,label
0,1202590843006,3.000,353.0,1.228867,8.900100,3.986968,0.008221,0.002269,-0.009966,1362.0,0.000000,1
1,1202590843006,3.033,77.0,1.137217,8.975278,3.981050,0.021118,-0.022515,0.024522,541.0,0.000000,1
2,1202590843006,6.173,283.0,1.522818,7.947064,5.075056,0.137567,-0.297055,-0.017418,154.0,2.863286,1
3,1202590843006,3.014,353.0,1.134775,8.932489,3.929421,-0.007959,-0.020543,-0.022986,1384.0,0.000000,1
4,1202590843006,5.209,340.0,0.858197,9.078037,2.944571,0.061924,0.015219,0.025517,1142.0,9.750134,1


In [7]:
df.shape

(16154418, 12)

In [8]:
import numpy as np
cols = ['bookingID','Accuracy','Bearing', 'acceleration_x', 'acceleration_y', 'acceleration_z',
'gyro_x', 'gyro_y', 'gyro_z', 'second', 'Speed']
df[cols] = df[cols].astype(np.int64)



In [9]:
df.head()

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed,label
0,1202590843006,3,353,1,8,3,0,0,0,1362,0,1
1,1202590843006,3,77,1,8,3,0,0,0,541,0,1
2,1202590843006,6,283,1,7,5,0,0,0,154,2,1
3,1202590843006,3,353,1,8,3,0,0,0,1384,0,1
4,1202590843006,5,340,0,9,2,0,0,0,1142,9,1


In [10]:
#Logic- Dangerous driving involves looking at acceleration and bearing data. If there is a high acceleration and high change in 
#bearing then you have a case of sharp turns at high speeds--> Dangerous

 

df['accelerationx2']= ((df['acceleration_x']**2) + (df['acceleration_y']**2) + (df['acceleration_z']**2))

df['resultant_acc'] = df['accelerationx2']**0.5 

df['MA_acc'] = df.rolling(window=2)['resultant_acc'].mean()

df['MA_acc_max']= df.groupby('bookingID')['MA_acc'].transform('max')
df['MA_acc_mean']= df.groupby('bookingID')['MA_acc'].transform('mean')
df['MA_acc_skew']= abs(df.groupby('bookingID')['MA_acc'].transform('skew'))
df['MA_acc_var']= df.groupby('bookingID')['MA_acc'].transform('var')


#Check if speed > speed limit. Max speed limit in Singapore is 80-90 km/h ` 25m/s
df['Speedmax'] = df.groupby('bookingID')['Speed'].transform('max')
df['Speedabovelimit'] = df['Speedmax'].apply(lambda x: 1 if x >25 else 0)

#change in bearing

df['bearing_chg'] = df.rolling(window=2)['Bearing'].mean() #assumption all bearings are vs true north measure clockwise"
df['bearing_chg_max']= df.groupby('bookingID')['bearing_chg'].transform('max')
df['bearing_chg_var']= df.groupby('bookingID')['bearing_chg'].transform('var')


#combination of acceleration and bearing - rudimenatary feature creation. looking for high acceleration and high bearing
df['bearing_acc_comb1'] = df['MA_acc_max'] +  df['bearing_chg_max']
df['bearing_acc_comb2'] = df['MA_acc_var'] +  df['bearing_chg_var']

In [11]:
df.columns

Index(['bookingID', 'Accuracy', 'Bearing', 'acceleration_x', 'acceleration_y',
       'acceleration_z', 'gyro_x', 'gyro_y', 'gyro_z', 'second', 'Speed',
       'label', 'accelerationx2', 'resultant_acc', 'MA_acc', 'MA_acc_max',
       'MA_acc_mean', 'MA_acc_skew', 'MA_acc_var', 'Speedmax',
       'Speedabovelimit', 'bearing_chg', 'bearing_chg_max', 'bearing_chg_var',
       'bearing_acc_comb1', 'bearing_acc_comb2'],
      dtype='object')

In [12]:
df.head()

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,...,MA_acc_mean,MA_acc_skew,MA_acc_var,Speedmax,Speedabovelimit,bearing_chg,bearing_chg_max,bearing_chg_var,bearing_acc_comb1,bearing_acc_comb2
0,1202590843006,3,353,1,8,3,0,0,0,1362,...,9.303264,0.179498,0.213838,18,0,NaN,358.5,10369.366595,369.774303,10369.580433
1,1202590843006,3,77,1,8,3,0,0,0,541,...,9.303264,0.179498,0.213838,18,0,215.0,358.5,10369.366595,369.774303,10369.580433
2,1202590843006,6,283,1,7,5,0,0,0,154,...,9.303264,0.179498,0.213838,18,0,180.0,358.5,10369.366595,369.774303,10369.580433
3,1202590843006,3,353,1,8,3,0,0,0,1384,...,9.303264,0.179498,0.213838,18,0,318.0,358.5,10369.366595,369.774303,10369.580433
4,1202590843006,5,340,0,9,2,0,0,0,1142,...,9.303264,0.179498,0.213838,18,0,346.5,358.5,10369.366595,369.774303,10369.580433


In [13]:
df = df.drop_duplicates(subset='bookingID', keep='last')

In [14]:
df.head()

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,...,MA_acc_mean,MA_acc_skew,MA_acc_var,Speedmax,Speedabovelimit,bearing_chg,bearing_chg_max,bearing_chg_var,bearing_acc_comb1,bearing_acc_comb2
1150,1202590843006,4,355,1,7,3,0,0,0,1448,...,9.303264,0.179498,0.213838,18,0,177.5,358.5,10369.366595,369.774303,10369.580433
2760,274877907034,3,78,0,9,3,0,0,0,923,...,9.210759,0.258107,0.141611,28,1,55.5,330.0,2692.820719,340.805594,2692.962331
4437,884763263056,3,89,0,9,2,0,0,0,1329,...,9.258184,0.213999,0.172664,26,1,199.0,312.5,3699.066355,324.399755,3699.239019
5481,1073741824054,4,168,1,8,0,0,0,0,1004,...,9.256674,1.334064,0.285676,19,0,238.0,308.0,2567.056543,321.161277,2567.342219
6381,1056561954943,3,57,0,10,-1,0,0,0,805,...,9.330645,0.215441,0.486080,24,0,75.5,358.5,6774.429718,370.464194,6774.915798


In [15]:
df.isnull().values.any() #there shouldnt be any null values

False

In [16]:
from sklearn.preprocessing import label_binarize

df['label']= label_binarize(df['label'], classes=[0,1])

In [17]:
df.shape

(20000, 26)

In [18]:
df = df.drop(['bookingID','Accuracy', 'Bearing', 'acceleration_x', 'acceleration_y',
       'acceleration_z', 'gyro_x', 'gyro_y', 'gyro_z', 'second', 'Speed','Speedmax','accelerationx2','MA_acc','resultant_acc'], axis=1)


In [19]:
df = df.reindex(np.random.permutation(df.index))

In [20]:
df.columns

Index(['label', 'MA_acc_max', 'MA_acc_mean', 'MA_acc_skew', 'MA_acc_var',
       'Speedabovelimit', 'bearing_chg', 'bearing_chg_max', 'bearing_chg_var',
       'bearing_acc_comb1', 'bearing_acc_comb2'],
      dtype='object')

In [21]:
df.head()

,label,MA_acc_max,MA_acc_mean,MA_acc_skew,MA_acc_var,Speedabovelimit,bearing_chg,bearing_chg_max,bearing_chg_var,bearing_acc_comb1,bearing_acc_comb2
14580280,0,11.302916,9.661478,0.035118,0.214766,0,118.0,292.0,4847.562777,303.302916,4847.777543
15553178,1,10.978767,9.235348,0.054441,0.254354,1,74.0,317.5,3695.117675,328.478767,3695.372029
2777767,0,11.314735,9.220159,0.160439,0.158036,0,0.0,357.0,6351.107210,368.314735,6351.265246
7868247,1,11.272871,9.148209,1.337905,0.112046,0,192.5,349.5,8672.912829,360.772871,8673.024875
12094255,1,12.063214,9.490504,0.157834,0.331377,0,59.5,317.5,4172.946725,329.563214,4173.278102


In [22]:
import seaborn as sns
import numpy as np

corr = df.corr()
sns.heatmap(corr)

In [23]:
from sklearn.model_selection import train_test_split
import numpy as np 
labels = df.label
features = df.drop('label', axis=1)


In [24]:
# split data into train and test sets
seed = 7
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=test_size, random_state=seed)

In [25]:
print('X_train Shape:', X_train.shape)
print('y_train Shape:', y_train.shape)
print('X_test Shape:', X_test.shape)
print('y_test Labels Shape:', y_test.shape)

X_train Shape: (16000, 10)
y_train Shape: (16000,)
X_test Shape: (4000, 10)
y_test Labels Shape: (4000,)


In [26]:
from imblearn.over_sampling import SMOTE
smt = SMOTE()
X_train, y_train = smt.fit_sample(X_train, y_train)

In [27]:
np.bincount(y_train)

array([12008, 12008], dtype=int64)

In [28]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000)
# Train the model on training data
rf.fit(X_train, y_train);

In [29]:
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
y_pred_class = rf.predict(X_test)
y_pred_prob = rf.predict_proba(X_test)[:, 1]
y_pred_prob
metrics.roc_auc_score(y_test, y_pred_class)

0.5309871684413505

In [30]:
from pprint import pprint
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1000,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


In [32]:
#Hyperparameter tuning for random forest
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start =500, stop = 2000, num = 4)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [40,80,120]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)


{'bootstrap': [True, False],
 'max_depth': [40, 80, 120, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [500, 1000, 1500, 2000]}


In [33]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, limited due to computational issues
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 26.6min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 117.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 204.3min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [500, 1000, 1500, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [40, 80, 120, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [34]:
rf_random.best_params_ #output: {'n_estimators': 1400, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 40, 'bootstrap': False}

{'n_estimators': 500,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': None,
 'bootstrap': False}

In [35]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rfhyper = RandomForestClassifier(n_estimators = 500, min_samples_split = 2, min_samples_leaf = 1, max_features = 'auto', max_depth = None, bootstrap = False)
# Train the model on training data
rfhyper.fit(X_train, y_train);

In [36]:
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
y_pred_class = rfhyper.predict(X_test)
y_pred_prob = rfhyper.predict_proba(X_test)[:, 1]
y_pred_prob
metrics.roc_auc_score(y_test, y_pred_class)

0.5253143846945914

In [39]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
param_grid = {'C': [ 0.00001, 0.0001, 0.001, 0.01,0.1, 1, 10, 100]}
LRmodel = RandomizedSearchCV(LogisticRegression(), param_grid, cv=3, scoring='roc_auc')

LRmodel.fit(X_train, y_train)

from sklearn import metrics
from sklearn.metrics import roc_curve, auc
y_pred_class = LRmodel.predict(X_test)
y_pred_prob = LRmodel.predict_proba(X_test)[:, 1]
y_pred_prob
metrics.roc_auc_score(y_test, y_pred_class)

C:\Users\natas\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:271: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
C:\Users\natas\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\natas\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\natas\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\natas\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solv

0.5543148947014325

In [40]:
from pprint import pprint
print('Parameters currently in use:\n')
pprint(LRmodel.get_params())

Parameters currently in use:

{'cv': 3,
 'error_score': 'raise-deprecating',
 'estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
 'estimator__C': 1.0,
 'estimator__class_weight': None,
 'estimator__dual': False,
 'estimator__fit_intercept': True,
 'estimator__intercept_scaling': 1,
 'estimator__max_iter': 100,
 'estimator__multi_class': 'warn',
 'estimator__n_jobs': None,
 'estimator__penalty': 'l2',
 'estimator__random_state': None,
 'estimator__solver': 'warn',
 'estimator__tol': 0.0001,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'fit_params': None,
 'iid': 'warn',
 'n_iter': 10,
 'n_jobs': None,
 'param_distributions': {'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]},
 'pre_dispatch': '2*n_jobs',
 'random_state': None,
 'refit': True,
 're

In [41]:
import xgboost as xgb

In [42]:
# split data into train and test sets
seed = 7
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=test_size, random_state=seed)

In [43]:
xg_class = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [44]:
#XGBoost base model
from numpy import loadtxt
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [45]:
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
y_pred_class = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)[:, 1]
y_pred_prob
metrics.roc_auc_score(y_test, y_pred_class)

0.525030361918066

In [46]:
#Hyperparameters for XGBoost
random_grid ={
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [47]:
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='reg:logistic',
                    silent=True, nthread=1)

In [48]:
#Hyperparameter tuning for XGBoost
from sklearn.model_selection import RandomizedSearchCV
from numpy import loadtxt
from xgboost import XGBClassifier

random_search = RandomizedSearchCV(xgb, param_distributions=random_grid, n_iter=5, scoring='roc_auc', n_jobs=4, cv=3, verbose=3, random_state=1001 )
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  15 out of  15 | elapsed:   40.9s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=600,
       n_jobs=1, nthread=1, objective='reg:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid='warn', n_iter=5, n_jobs=4,
          param_distributions={'min_child_weight': [1, 5, 10], 'gamma': [0.5, 1.5, 2, 5], 'subsample': [0.6, 0.8, 1.0], 'colsample_bytree': [0.6, 0.8, 1.0], 'max_depth': [3, 4, 5]},
          pre_dispatch='2*n_jobs', random_state=1001, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=3)

In [52]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (3, 5))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)


 All results:
{'mean_fit_time': array([12.11959036,  6.48532152,  9.59068513,  7.57208061,  5.40122255]), 'std_fit_time': array([0.02831515, 0.27565809, 0.07363741, 0.04703692, 0.84195478]), 'mean_score_time': array([0.26130191, 0.14461374, 0.24933314, 0.20079652, 0.16523306]), 'std_score_time': array([0.00141051, 0.00141023, 0.00293599, 0.00463064, 0.01851271]), 'param_subsample': masked_array(data=[0.6, 0.6, 0.6, 0.6, 0.6],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[1, 1, 5, 10, 10],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[5, 3, 5, 4, 4],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[1.5, 2, 0.5, 2, 0.5],
             mask=[False, False, False, False, False],
       fill_value='?',
   

In [53]:
modeltuned = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=2, learning_rate=0.02, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=600,
       n_jobs=1, nthread=1, objective='reg:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.6)
modeltuned.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=2, learning_rate=0.02, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=600,
       n_jobs=1, nthread=1, objective='reg:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.6)

In [54]:
y_pred_class = modeltuned.predict(X_test)
y_pred_prob = modeltuned.predict_proba(X_test)[:, 1]
y_pred_prob
metrics.roc_auc_score(y_test, y_pred_class)

0.526364585178314